# Policy plots

In [1]:
from huggingface_hub import hf_hub_download, HfApi
from matplotlib.colors import ListedColormap as LCM
from plotnine import ggplot, aes, geom_point, geom_line, facet_grid, scale_color_manual, ylim
from skopt import load
from stable_baselines3 import PPO

from rl4fisheries import AsmEnv, FMsy, ConstantEscapement, PrecautionaryPrinciple
from rl4fisheries.utils import evaluate_agent
from rl4fisheries.envs.asm_fns import get_r_devs_logn_unif as get_r_devs

import numpy as np
import pandas as pd
import ray

In [2]:
_viridis_data = [[0.267004, 0.004874, 0.329415],
                 [0.268510, 0.009605, 0.335427],
                 [0.269944, 0.014625, 0.341379],
                 [0.271305, 0.019942, 0.347269],
                 [0.272594, 0.025563, 0.353093],
                 [0.273809, 0.031497, 0.358853],
                 [0.274952, 0.037752, 0.364543],
                 [0.276022, 0.044167, 0.370164],
                 [0.277018, 0.050344, 0.375715],
                 [0.277941, 0.056324, 0.381191],
                 [0.278791, 0.062145, 0.386592],
                 [0.279566, 0.067836, 0.391917],
                 [0.280267, 0.073417, 0.397163],
                 [0.280894, 0.078907, 0.402329],
                 [0.281446, 0.084320, 0.407414],
                 [0.281924, 0.089666, 0.412415],
                 [0.282327, 0.094955, 0.417331],
                 [0.282656, 0.100196, 0.422160],
                 [0.282910, 0.105393, 0.426902],
                 [0.283091, 0.110553, 0.431554],
                 [0.283197, 0.115680, 0.436115],
                 [0.283229, 0.120777, 0.440584],
                 [0.283187, 0.125848, 0.444960],
                 [0.283072, 0.130895, 0.449241],
                 [0.282884, 0.135920, 0.453427],
                 [0.282623, 0.140926, 0.457517],
                 [0.282290, 0.145912, 0.461510],
                 [0.281887, 0.150881, 0.465405],
                 [0.281412, 0.155834, 0.469201],
                 [0.280868, 0.160771, 0.472899],
                 [0.280255, 0.165693, 0.476498],
                 [0.279574, 0.170599, 0.479997],
                 [0.278826, 0.175490, 0.483397],
                 [0.278012, 0.180367, 0.486697],
                 [0.277134, 0.185228, 0.489898],
                 [0.276194, 0.190074, 0.493001],
                 [0.275191, 0.194905, 0.496005],
                 [0.274128, 0.199721, 0.498911],
                 [0.273006, 0.204520, 0.501721],
                 [0.271828, 0.209303, 0.504434],
                 [0.270595, 0.214069, 0.507052],
                 [0.269308, 0.218818, 0.509577],
                 [0.267968, 0.223549, 0.512008],
                 [0.266580, 0.228262, 0.514349],
                 [0.265145, 0.232956, 0.516599],
                 [0.263663, 0.237631, 0.518762],
                 [0.262138, 0.242286, 0.520837],
                 [0.260571, 0.246922, 0.522828],
                 [0.258965, 0.251537, 0.524736],
                 [0.257322, 0.256130, 0.526563],
                 [0.255645, 0.260703, 0.528312],
                 [0.253935, 0.265254, 0.529983],
                 [0.252194, 0.269783, 0.531579],
                 [0.250425, 0.274290, 0.533103],
                 [0.248629, 0.278775, 0.534556],
                 [0.246811, 0.283237, 0.535941],
                 [0.244972, 0.287675, 0.537260],
                 [0.243113, 0.292092, 0.538516],
                 [0.241237, 0.296485, 0.539709],
                 [0.239346, 0.300855, 0.540844],
                 [0.237441, 0.305202, 0.541921],
                 [0.235526, 0.309527, 0.542944],
                 [0.233603, 0.313828, 0.543914],
                 [0.231674, 0.318106, 0.544834],
                 [0.229739, 0.322361, 0.545706],
                 [0.227802, 0.326594, 0.546532],
                 [0.225863, 0.330805, 0.547314],
                 [0.223925, 0.334994, 0.548053],
                 [0.221989, 0.339161, 0.548752],
                 [0.220057, 0.343307, 0.549413],
                 [0.218130, 0.347432, 0.550038],
                 [0.216210, 0.351535, 0.550627],
                 [0.214298, 0.355619, 0.551184],
                 [0.212395, 0.359683, 0.551710],
                 [0.210503, 0.363727, 0.552206],
                 [0.208623, 0.367752, 0.552675],
                 [0.206756, 0.371758, 0.553117],
                 [0.204903, 0.375746, 0.553533],
                 [0.203063, 0.379716, 0.553925],
                 [0.201239, 0.383670, 0.554294],
                 [0.199430, 0.387607, 0.554642],
                 [0.197636, 0.391528, 0.554969],
                 [0.195860, 0.395433, 0.555276],
                 [0.194100, 0.399323, 0.555565],
                 [0.192357, 0.403199, 0.555836],
                 [0.190631, 0.407061, 0.556089],
                 [0.188923, 0.410910, 0.556326],
                 [0.187231, 0.414746, 0.556547],
                 [0.185556, 0.418570, 0.556753],
                 [0.183898, 0.422383, 0.556944],
                 [0.182256, 0.426184, 0.557120],
                 [0.180629, 0.429975, 0.557282],
                 [0.179019, 0.433756, 0.557430],
                 [0.177423, 0.437527, 0.557565],
                 [0.175841, 0.441290, 0.557685],
                 [0.174274, 0.445044, 0.557792],
                 [0.172719, 0.448791, 0.557885],
                 [0.171176, 0.452530, 0.557965],
                 [0.169646, 0.456262, 0.558030],
                 [0.168126, 0.459988, 0.558082],
                 [0.166617, 0.463708, 0.558119],
                 [0.165117, 0.467423, 0.558141],
                 [0.163625, 0.471133, 0.558148],
                 [0.162142, 0.474838, 0.558140],
                 [0.160665, 0.478540, 0.558115],
                 [0.159194, 0.482237, 0.558073],
                 [0.157729, 0.485932, 0.558013],
                 [0.156270, 0.489624, 0.557936],
                 [0.154815, 0.493313, 0.557840],
                 [0.153364, 0.497000, 0.557724],
                 [0.151918, 0.500685, 0.557587],
                 [0.150476, 0.504369, 0.557430],
                 [0.149039, 0.508051, 0.557250],
                 [0.147607, 0.511733, 0.557049],
                 [0.146180, 0.515413, 0.556823],
                 [0.144759, 0.519093, 0.556572],
                 [0.143343, 0.522773, 0.556295],
                 [0.141935, 0.526453, 0.555991],
                 [0.140536, 0.530132, 0.555659],
                 [0.139147, 0.533812, 0.555298],
                 [0.137770, 0.537492, 0.554906],
                 [0.136408, 0.541173, 0.554483],
                 [0.135066, 0.544853, 0.554029],
                 [0.133743, 0.548535, 0.553541],
                 [0.132444, 0.552216, 0.553018],
                 [0.131172, 0.555899, 0.552459],
                 [0.129933, 0.559582, 0.551864],
                 [0.128729, 0.563265, 0.551229],
                 [0.127568, 0.566949, 0.550556],
                 [0.126453, 0.570633, 0.549841],
                 [0.125394, 0.574318, 0.549086],
                 [0.124395, 0.578002, 0.548287],
                 [0.123463, 0.581687, 0.547445],
                 [0.122606, 0.585371, 0.546557],
                 [0.121831, 0.589055, 0.545623],
                 [0.121148, 0.592739, 0.544641],
                 [0.120565, 0.596422, 0.543611],
                 [0.120092, 0.600104, 0.542530],
                 [0.119738, 0.603785, 0.541400],
                 [0.119512, 0.607464, 0.540218],
                 [0.119423, 0.611141, 0.538982],
                 [0.119483, 0.614817, 0.537692],
                 [0.119699, 0.618490, 0.536347],
                 [0.120081, 0.622161, 0.534946],
                 [0.120638, 0.625828, 0.533488],
                 [0.121380, 0.629492, 0.531973],
                 [0.122312, 0.633153, 0.530398],
                 [0.123444, 0.636809, 0.528763],
                 [0.124780, 0.640461, 0.527068],
                 [0.126326, 0.644107, 0.525311],
                 [0.128087, 0.647749, 0.523491],
                 [0.130067, 0.651384, 0.521608],
                 [0.132268, 0.655014, 0.519661],
                 [0.134692, 0.658636, 0.517649],
                 [0.137339, 0.662252, 0.515571],
                 [0.140210, 0.665859, 0.513427],
                 [0.143303, 0.669459, 0.511215],
                 [0.146616, 0.673050, 0.508936],
                 [0.150148, 0.676631, 0.506589],
                 [0.153894, 0.680203, 0.504172],
                 [0.157851, 0.683765, 0.501686],
                 [0.162016, 0.687316, 0.499129],
                 [0.166383, 0.690856, 0.496502],
                 [0.170948, 0.694384, 0.493803],
                 [0.175707, 0.697900, 0.491033],
                 [0.180653, 0.701402, 0.488189],
                 [0.185783, 0.704891, 0.485273],
                 [0.191090, 0.708366, 0.482284],
                 [0.196571, 0.711827, 0.479221],
                 [0.202219, 0.715272, 0.476084],
                 [0.208030, 0.718701, 0.472873],
                 [0.214000, 0.722114, 0.469588],
                 [0.220124, 0.725509, 0.466226],
                 [0.226397, 0.728888, 0.462789],
                 [0.232815, 0.732247, 0.459277],
                 [0.239374, 0.735588, 0.455688],
                 [0.246070, 0.738910, 0.452024],
                 [0.252899, 0.742211, 0.448284],
                 [0.259857, 0.745492, 0.444467],
                 [0.266941, 0.748751, 0.440573],
                 [0.274149, 0.751988, 0.436601],
                 [0.281477, 0.755203, 0.432552],
                 [0.288921, 0.758394, 0.428426],
                 [0.296479, 0.761561, 0.424223],
                 [0.304148, 0.764704, 0.419943],
                 [0.311925, 0.767822, 0.415586],
                 [0.319809, 0.770914, 0.411152],
                 [0.327796, 0.773980, 0.406640],
                 [0.335885, 0.777018, 0.402049],
                 [0.344074, 0.780029, 0.397381],
                 [0.352360, 0.783011, 0.392636],
                 [0.360741, 0.785964, 0.387814],
                 [0.369214, 0.788888, 0.382914],
                 [0.377779, 0.791781, 0.377939],
                 [0.386433, 0.794644, 0.372886],
                 [0.395174, 0.797475, 0.367757],
                 [0.404001, 0.800275, 0.362552],
                 [0.412913, 0.803041, 0.357269],
                 [0.421908, 0.805774, 0.351910],
                 [0.430983, 0.808473, 0.346476],
                 [0.440137, 0.811138, 0.340967],
                 [0.449368, 0.813768, 0.335384],
                 [0.458674, 0.816363, 0.329727],
                 [0.468053, 0.818921, 0.323998],
                 [0.477504, 0.821444, 0.318195],
                 [0.487026, 0.823929, 0.312321],
                 [0.496615, 0.826376, 0.306377],
                 [0.506271, 0.828786, 0.300362],
                 [0.515992, 0.831158, 0.294279],
                 [0.525776, 0.833491, 0.288127],
                 [0.535621, 0.835785, 0.281908],
                 [0.545524, 0.838039, 0.275626],
                 [0.555484, 0.840254, 0.269281],
                 [0.565498, 0.842430, 0.262877],
                 [0.575563, 0.844566, 0.256415],
                 [0.585678, 0.846661, 0.249897],
                 [0.595839, 0.848717, 0.243329],
                 [0.606045, 0.850733, 0.236712],
                 [0.616293, 0.852709, 0.230052],
                 [0.626579, 0.854645, 0.223353],
                 [0.636902, 0.856542, 0.216620],
                 [0.647257, 0.858400, 0.209861],
                 [0.657642, 0.860219, 0.203082],
                 [0.668054, 0.861999, 0.196293],
                 [0.678489, 0.863742, 0.189503],
                 [0.688944, 0.865448, 0.182725],
                 [0.699415, 0.867117, 0.175971],
                 [0.709898, 0.868751, 0.169257],
                 [0.720391, 0.870350, 0.162603],
                 [0.730889, 0.871916, 0.156029],
                 [0.741388, 0.873449, 0.149561],
                 [0.751884, 0.874951, 0.143228],
                 [0.762373, 0.876424, 0.137064],
                 [0.772852, 0.877868, 0.131109],
                 [0.783315, 0.879285, 0.125405],
                 [0.793760, 0.880678, 0.120005],
                 [0.804182, 0.882046, 0.114965],
                 [0.814576, 0.883393, 0.110347],
                 [0.824940, 0.884720, 0.106217],
                 [0.835270, 0.886029, 0.102646],
                 [0.845561, 0.887322, 0.099702],
                 [0.855810, 0.888601, 0.097452],
                 [0.866013, 0.889868, 0.095953],
                 [0.876168, 0.891125, 0.095250],
                 [0.886271, 0.892374, 0.095374],
                 [0.896320, 0.893616, 0.096335],
                 [0.906311, 0.894855, 0.098125],
                 [0.916242, 0.896091, 0.100717],
                 [0.926106, 0.897330, 0.104071],
                 [0.935904, 0.898570, 0.108131],
                 [0.945636, 0.899815, 0.112838],
                 [0.955300, 0.901065, 0.118128],
                 [0.964894, 0.902323, 0.123941],
                 [0.974417, 0.903590, 0.130215],
                 [0.983868, 0.904867, 0.136897],
                 [0.993248, 0.906157, 0.143936]]

In [3]:
temp_env = AsmEnv()
min_wt = temp_env.parameters['min_wt']
max_wt = temp_env.parameters['max_wt']

MWT_OBS_LIST_SHORT = (
    - 1 + 2 * (
        (
            (np.linspace(start=0.25, stop=0.81, num=8) - min_wt) /
            (max_wt - min_wt)
        )
    )
)

# formula
# -1 + 2 * (wt-min_wt) / (max_wt - min_wt)

# old:
# [0, 0.2, 0.3, 0.4, 0.45, 0.5, 0.55, 0.9]

def rgb_to_hex(rgb):
    return '#%02x%02x%02x' % tuple(rgb)

viridis_vals = [
    _viridis_data[
        10 + ((len(_viridis_data)-10) // (len(MWT_OBS_LIST_SHORT)+1)) * i
    ]
    for i in range(len(MWT_OBS_LIST_SHORT))
]

color_rgbs = [
    [255,0,255], # 'black' ,
    *[[int(comp) for comp in color] for color in 255 * np.array(viridis_vals)],
    [255, 188, 0], # 'orange', 
    [150, 150, 150], # 'gray', 
    [255, 43, 0], # 'red',  
    ]

viridis_rgbs = [
    *[[int(comp) for comp in color] for color in 255 * np.array(viridis_vals)],
]
other_rgbs = [ 
    [255,0,255], # 'rose' , 
    [255, 43, 0], # 'red',  
    [150, 150, 150], # 'gray',
    [255, 188, 0], # 'orange',
]

color_hexs = [rgb_to_hex(rgb) for rgb in color_rgbs]

viridis_hexs = [rgb_to_hex(rgb) for rgb in viridis_rgbs]
other_hexs = [rgb_to_hex(rgb) for rgb in other_rgbs]

my_cmap = LCM(
    color_rgbs
)

In [4]:
## UM1

CFG_UM1_2o = {
    'observation_fn_id': 'observe_2o',
    'n_observs': 2,
    'obs_noise':0.1,
    #
    'harvest_fn_name': "default",
    'upow': 1,
}
CFG_UM1_bm = {
    'observation_fn_id': 'observe_1o',
    'n_observs': 1,
    'obs_noise':0.1,
    #
    'harvest_fn_name': "default",
    'upow': 1,
}

## UM2

CFG_UM2_2o = {
    'observation_fn_id': 'observe_2o',
    'n_observs': 2,
    'obs_noise':0.1,
    #
    'harvest_fn_name': "default",
    'upow': 0.6,
}
CFG_UM2_bm = {
    'observation_fn_id': 'observe_1o',
    'n_observs': 1,
    'obs_noise':0.1,
    #
    'harvest_fn_name': "default",
    'upow': 0.6,
}

## UM3

CFG_UM3_2o = {
    'observation_fn_id': 'observe_2o',
    'n_observs': 2,
    'obs_noise':0.1,
    #
    'harvest_fn_name': "trophy",
    'upow': 1,
    'n_trophy_ages': 10
}
CFG_UM3_bm = {
    'observation_fn_id': 'observe_1o',
    'n_observs': 1,
    'obs_noise':0.1,
    #
    'harvest_fn_name': "trophy",
    'upow': 1,
    'n_trophy_ages': 10
}

## Load

In [5]:
cr_UM1_file = hf_hub_download(repo_id="boettiger-lab/rl4eco", filename="sb3/rl4fisheries/post-review-results/cr-UM1-noise01.pkl")
cr_UM2_file = hf_hub_download(repo_id="boettiger-lab/rl4eco", filename="sb3/rl4fisheries/post-review-results/cr-UM2-noise01.pkl")
cr_UM3_file = hf_hub_download(repo_id="boettiger-lab/rl4eco", filename="sb3/rl4fisheries/post-review-results/cr-UM3-noise01.pkl")

msy_UM1_file = hf_hub_download(repo_id="boettiger-lab/rl4eco", filename="sb3/rl4fisheries/post-review-results/msy-UM1-noise01.pkl")
msy_UM2_file = hf_hub_download(repo_id="boettiger-lab/rl4eco", filename="sb3/rl4fisheries/post-review-results/msy-UM2-noise01.pkl")
msy_UM3_file = hf_hub_download(repo_id="boettiger-lab/rl4eco", filename="sb3/rl4fisheries/post-review-results/msy-UM3-noise01.pkl")

In [6]:
cr_UM1 = load(cr_UM1_file)
cr_UM2 = load(cr_UM2_file)
cr_UM3 = load(cr_UM3_file)

msy_UM1 = load(msy_UM1_file)
msy_UM2 = load(msy_UM2_file)
msy_UM3 = load(msy_UM3_file)

In [7]:
def from_radius_theta(radius, theta, y2):
    x1 = radius * np.sin(theta)
    x2 = radius * np.cos(theta)
    return {'x1': x1, 'x2': x2, 'y2': y2}

cr_UM1_cartesian = from_radius_theta(*cr_UM1.x)

In [8]:
cr_UM2_cartesian = from_radius_theta(*cr_UM2.x)

In [9]:
cr_UM3_cartesian = from_radius_theta(*cr_UM3.x)

In [10]:
fmsy_UM1 = msy_UM1.x[0]
fmsy_UM2 = msy_UM2.x[0]
fmsy_UM3 = msy_UM3.x[0]

msy_rew_UM1 = -msy_UM1.fun
msy_rew_UM2 = -msy_UM2.fun
msy_rew_UM3 = -msy_UM3.fun

print(f"UM1: fmsy = {fmsy_UM1}, rew = {msy_rew_UM1}")
print(f"UM2: fmsy = {fmsy_UM2}, rew = {msy_rew_UM2}")
print(f"UM3: fmsy = {fmsy_UM3}, rew = {msy_rew_UM3}")

UM1: fmsy = 0.07136400999138845, rew = 74.57189825941235
UM2: fmsy = 0.0603909560251637, rew = 177.83347764121123
UM3: fmsy = 0.05994460173147967, rew = 36.42077652223719


In [11]:
base_fname = "sb3/rl4fisheries/post-review-results/PPO-AsmEnv-"
repo = "boettiger-lab/rl4eco"

PPO_2o_UM1_file = hf_hub_download(repo_id=repo, filename=base_fname+"2obs-UM1-256-64-16-noise0.1-chkpnt2.zip")
PPO_bm_UM1_file = hf_hub_download(repo_id=repo, filename=base_fname+"biomass-UM1-64-32-16-noise0.1-chkpnt2.zip")

PPO_2o_UM2_file = hf_hub_download(repo_id=repo, filename=base_fname+"2obs-UM2-256-64-16-noise0.1-chkpnt4.zip")
PPO_bm_UM2_file = hf_hub_download(repo_id=repo, filename=base_fname+"biomass-UM2-64-32-16-noise0.1-chkpnt4.zip")

PPO_2o_UM3_file = hf_hub_download(repo_id=repo, filename=base_fname+"2obs-UM3-256-64-16-noise0.1-chkpnt2.zip")
PPO_bm_UM3_file = hf_hub_download(repo_id=repo, filename=base_fname+"biomass-UM3-64-32-16-noise0.1-chkpnt4.zip")




In [12]:
PPO_2o_UM1 = PPO.load(PPO_2o_UM1_file, device='cpu')
PPO_bm_UM1 = PPO.load(PPO_bm_UM1_file, device='cpu')

PPO_2o_UM2 = PPO.load(PPO_2o_UM2_file, device='cpu')
PPO_bm_UM2 = PPO.load(PPO_bm_UM2_file, device='cpu')

PPO_2o_UM3 = PPO.load(PPO_2o_UM3_file, device='cpu')
PPO_bm_UM3 = PPO.load(PPO_bm_UM3_file, device='cpu')

## Policy plot utilities

In [13]:
from itertools import product

def obs_to_mwt(obs, asm_env):
    return asm_env.parameters['min_wt'] + (
        asm_env.parameters['max_wt'] - asm_env.parameters['min_wt']
    ) * (obs + 1) / 2

def obs_to_bms(obs, asm_env):
    return asm_env.bound * (obs + 1) / 2

def get_mwt_policy(mwt_obs_list, *, agent, asm_env):
    mwt_list = [obs_to_mwt(mwt, asm_env) for mwt in mwt_obs_list]
    return {
        'mwt': mwt_list, 
        'fishing_intensity': [ 
            0.5 * (1 + agent.predict(np.float32([mwt]))[0][0]) 
            for mwt in mwt_obs_list
        ],
    }

def get_bms_policy(bms_obs_list, *, agent, asm_env):
    bms_list = [obs_to_bms(bms, asm_env) for bms in bms_obs_list]
    return {
        'bms': bms_list, 
        'fishing_intensity': [ 
            0.5 * (1 + agent.predict(np.float32([bms]))[0][0]) 
            for bms in bms_obs_list
        ],
    }

def get_2obs_policy(bms_obs_list, mwt_obs_list, *, agent, asm_env):
    mwt_list = [obs_to_mwt(mwt, asm_env) for mwt in mwt_obs_list]
    bms_list = [obs_to_bms(bms, asm_env) for bms in bms_obs_list]

    predictors = list(product(bms_list, mwt_list))
    bms_list_long = [pred[0] for pred in predictors]
    mwt_list_long = [pred[1] for pred in predictors]
    
    return {
        'bms': bms_list_long, 
        'mwt': mwt_list_long,
        'fishing_intensity': [ 
            0.5 * (1 + agent.predict(np.float32([bms, mwt]))[0][0]) 
            for (bms, mwt) in product(bms_obs_list, mwt_obs_list)
        ],
    }

def from_radius_theta(radius, theta, y2):
    x1 = radius * np.sin(theta)
    x2 = radius * np.cos(theta)
    return {'x1': x1, 'x2': x2, 'y2': y2}

## UM1

In [14]:
bms_obs_list = np.linspace(-1, -1+0.14, 500)
mwt_obs_list = np.linspace(-1,1,500)


UM1_2o_pol = pd.DataFrame(get_2obs_policy(
    bms_obs_list, MWT_OBS_LIST_SHORT, 
    agent = PPO_2o_UM1, 
    asm_env = AsmEnv(config=CFG_UM1_2o),
)) 

UM1_bm_pol = pd.DataFrame(get_bms_policy(
    bms_obs_list, 
    agent = PPO_bm_UM1, 
    asm_env = AsmEnv(config=CFG_UM1_bm),
)) 

In [15]:
from_radius_theta(*cr_UM1.x)


{'x1': 0.36843859112290955, 'x2': 1.9251587862339785, 'y2': 0.8}

In [16]:
fmsy_avg_rewards = {
    'um1': msy_rew_UM1/1000,
    'um2': msy_rew_UM2/1000,
    'um3': msy_rew_UM3/1000,
}

fmsy_vals = {
    'um1': fmsy_UM1,
    'um2': fmsy_UM2,
    'um3': fmsy_UM3,
}

bmsy = fmsy_UM1 / fmsy_avg_rewards['um1']


def get_dfo_pol(fmsy, bmsy, env_cfg):
    return PrecautionaryPrinciple(
        env = AsmEnv(config=env_cfg),
        x1 = 0.4 * bmsy,
        x2 = 0.8 * bmsy,
        y2 = fmsy,
    )


UM1_cr_pol = pd.DataFrame(
    get_bms_policy(
        bms_obs_list,
        agent = PrecautionaryPrinciple(
            env = AsmEnv(config=CFG_UM1_bm),
            **(from_radius_theta(*cr_UM1.x)), 
        ),
        asm_env = AsmEnv(config=CFG_UM1_bm),
    )
)

UM1_DFO_pol = pd.DataFrame(
    get_bms_policy(
        bms_obs_list,
        agent = get_dfo_pol(
            fmsy=fmsy_vals['um1'],
            bmsy=bmsy,
            env_cfg=CFG_UM1_bm,
        ),
        asm_env = AsmEnv(config=CFG_UM1_bm),
    )
)


UM1_msy_pol = pd.DataFrame(get_bms_policy(
    bms_obs_list, 
    agent = FMsy(
        env = AsmEnv(config=CFG_UM1_bm), mortality=msy_UM1.x[0]
    ), 
    asm_env = AsmEnv(config=CFG_UM1_bm),
)) 

In [17]:
UM1_2o_pol['policy'] = UM1_2o_pol.apply(
    func = lambda row: f'2 obs. RL at m. wt. {row.mwt:.2f}',
    axis=1,
)
UM1_bm_pol['policy'] = '1 obs. RL'

UM1_cr_pol['policy'] = 'Opt. Prec. Pol.'
UM1_msy_pol['policy'] = 'Const. Mort.'
UM1_DFO_pol['policy'] = 'Constr. Prec. Pol.'

UM1_policies = pd.concat([
    UM1_msy_pol,
    UM1_DFO_pol,
    UM1_cr_pol,
    UM1_bm_pol,
    UM1_2o_pol[['bms', 'fishing_intensity', 'policy']],
])

UM1_policies['strategy_type'] = UM1_policies.apply(
    axis=1,
    func=lambda row: '2 Obs. RL' if row.policy[:9] == '2 obs. RL' else 'Other Policies',
)

In [33]:
# (
#     (
#         ggplot(UM1_policies[UM1_policies.strategy_type == "Other Policies"], aes(x='bms', y='fishing_intensity', color='policy')) 
#         + geom_point() 
#         + scale_color_manual(values=other_hexs)
#         # + facet_grid(rows = 'strategy_type')
#         + ylim(0,1)
#     ).save('plots/UM1_all_1o_policy_fns'),
#     (
#         ggplot(UM1_policies[UM1_policies.strategy_type == "2 Obs. RL"], aes(x='bms', y='fishing_intensity', color='policy')) 
#         + geom_point() 
#         + scale_color_manual(values=viridis_hexs)
#         # + facet_grid(rows = 'strategy_type')
#         + ylim(0,1)
#     ).save('plots/UM1_2o_policy_fns'),
# )

## UM2

In [19]:
from_radius_theta(*cr_UM2.x)


{'x1': 3.408451744085217e-05,
 'x2': 3.4084517439716016,
 'y2': 0.5019995089665347}

In [20]:
bms_obs_list = np.linspace(-1, -1+0.14, 500)
mwt_obs_list_short = MWT_OBS_LIST_SHORT
mwt_obs_list = np.linspace(-1,1,500)


UM2_2o_pol = pd.DataFrame(get_2obs_policy(
    bms_obs_list, mwt_obs_list_short, 
    agent = PPO_2o_UM2, 
    asm_env = AsmEnv(config=CFG_UM2_2o),
)) 

UM2_bm_pol = pd.DataFrame(get_bms_policy(
    bms_obs_list, 
    agent = PPO_bm_UM2, 
    asm_env = AsmEnv(config=CFG_UM2_bm),
)) 

In [21]:
UM2_cr_pol = pd.DataFrame(
    get_bms_policy(
        bms_obs_list,
        agent = PrecautionaryPrinciple(
            env = AsmEnv(config=CFG_UM2_bm),
            **(from_radius_theta(*cr_UM2.x)), 
        ),
        asm_env = AsmEnv(config=CFG_UM2_bm),
    )
)

UM2_DFO_pol = pd.DataFrame(
    get_bms_policy(
        bms_obs_list,
        agent = get_dfo_pol(
            fmsy=fmsy_vals['um2'],
            bmsy=bmsy,
            env_cfg=CFG_UM2_bm,
        ),
        asm_env = AsmEnv(config=CFG_UM2_bm),
    )
)


UM2_msy_pol = pd.DataFrame(get_bms_policy(
    bms_obs_list, 
    agent = FMsy(
        env = AsmEnv(config=CFG_UM2_bm), mortality=msy_UM2.x[0]
    ), 
    asm_env = AsmEnv(config=CFG_UM2_bm),
)) 

In [22]:
UM2_2o_pol['policy'] = UM2_2o_pol.apply(
    func = lambda row: f'2 obs. RL at m. wt. {row.mwt:.2f}',
    axis=1,
)
UM2_bm_pol['policy'] = '1 obs. RL'

UM2_cr_pol['policy'] = 'Opt. Prec. Pol.'
UM2_msy_pol['policy'] = 'Const. Mort.'
UM2_DFO_pol['policy'] = 'Constr. Prec. Pol.'

UM2_policies = pd.concat([
    UM2_msy_pol,
    UM2_DFO_pol,
    UM2_cr_pol,
    UM2_bm_pol,
    UM2_2o_pol[['bms', 'fishing_intensity', 'policy']],
])

UM2_policies['strategy_type'] = UM2_policies.apply(
    axis=1,
    func=lambda row: '2 Obs. RL' if row.policy[:9] == '2 obs. RL' else 'Other Policies',
)

In [23]:
# (
#     (
#         ggplot(UM2_policies[UM2_policies.strategy_type == "Other Policies"], aes(x='bms', y='fishing_intensity', color='policy')) 
#         + geom_point() 
#         + scale_color_manual(values=other_hexs)
#         # + facet_grid(rows = 'strategy_type')
#         + ylim(0,1)
#     ).save('plots/UM2_all_1o_policy_fns'),
#     (
#         ggplot(UM2_policies[UM2_policies.strategy_type == "2 Obs. RL"], aes(x='bms', y='fishing_intensity', color='policy')) 
#         + geom_point() 
#         + scale_color_manual(values=viridis_hexs)
#         # + facet_grid(rows = 'strategy_type')
#         + ylim(0,1)
#     ).save('plots/UM2_2o_policy_fns'),
# )

## UM3

In [24]:
from_radius_theta(*cr_UM3.x)


{'x1': 9.322532180398698e-05,
 'x2': 9.322532180087947,
 'y2': 0.5490475337202091}

In [25]:
bms_obs_list = np.linspace(-1, -1+0.14, 500)
mwt_obs_list_short = MWT_OBS_LIST_SHORT # [0, 0.4, 0.45, 0.5, 0.55, 0.9]
mwt_obs_list = np.linspace(-1,1,500)


UM3_2o_pol = pd.DataFrame(get_2obs_policy(
    bms_obs_list, mwt_obs_list_short, 
    agent = PPO_2o_UM3, 
    asm_env = AsmEnv(config=CFG_UM3_2o),
)) 

UM3_bm_pol = pd.DataFrame(get_bms_policy(
    bms_obs_list, 
    agent = PPO_bm_UM3, 
    asm_env = AsmEnv(config=CFG_UM3_bm),
)) 

In [26]:
UM3_cr_pol = pd.DataFrame(
    get_bms_policy(
        bms_obs_list,
        agent = PrecautionaryPrinciple(
            env = AsmEnv(config=CFG_UM3_bm),
            **(from_radius_theta(*cr_UM3.x)), 
        ),
        asm_env = AsmEnv(config=CFG_UM3_bm),
    )
)

UM3_DFO_pol = pd.DataFrame(
    get_bms_policy(
        bms_obs_list,
        agent = get_dfo_pol(
            fmsy=fmsy_vals['um3'],
            bmsy=bmsy,
            env_cfg=CFG_UM3_bm,
        ),
        asm_env = AsmEnv(config=CFG_UM3_bm),
    )
)

UM3_msy_pol = pd.DataFrame(get_bms_policy(
    bms_obs_list, 
    agent = FMsy(
        env = AsmEnv(config=CFG_UM3_bm), mortality=msy_UM3.x[0]
    ), 
    asm_env = AsmEnv(config=CFG_UM3_bm),
)) 

In [27]:
UM3_2o_pol['policy'] = UM3_2o_pol.apply(
    func = lambda row: f'2 obs. RL at m. wt. {row.mwt:.2f}',
    axis=1,
)
UM3_bm_pol['policy'] = '1 obs. RL'

UM3_cr_pol['policy'] = 'Opt. Prec. Pol.'
UM3_msy_pol['policy'] = 'Const. Mort.'
UM3_DFO_pol['policy'] = 'Constr. Prec. Pol.'


UM3_policies = pd.concat([
    UM3_msy_pol,
    UM3_DFO_pol,
    UM3_cr_pol,
    UM3_bm_pol,
    UM3_2o_pol[['bms', 'fishing_intensity', 'policy']],
])

UM3_policies['strategy_type'] = UM3_policies.apply(
    axis=1,
    func=lambda row: '2 Obs. RL' if row.policy[:9] == '2 obs. RL' else 'Other Policies',
)

In [28]:
# (
#     (
#         ggplot(UM3_policies[UM3_policies.strategy_type == "Other Policies"], aes(x='bms', y='fishing_intensity', color='policy')) 
#         + geom_point() 
#         + scale_color_manual(values=other_hexs)
#         # + facet_grid(rows = 'strategy_type')
#         + ylim(0,1)
#     ).save('plots/UM3_all_1o_policy_fns'),
#     (
#         ggplot(UM3_policies[UM3_policies.strategy_type == "2 Obs. RL"], aes(x='bms', y='fishing_intensity', color='policy')) 
#         + geom_point() 
#         + scale_color_manual(values=viridis_hexs)
#         # + facet_grid(rows = 'strategy_type')
#         + ylim(0,1)
#     ).save('plots/UM3_2o_policy_fns'),
# )

## Saving

In [29]:
UM1_policies.to_csv("csv/UM1_policies.csv")
UM2_policies.to_csv("csv/UM2_policies.csv")
UM3_policies.to_csv("csv/UM3_policies.csv")

In [30]:
(MWT_OBS_LIST_SHORT+1)/2

array([0.25099619, 0.3342019 , 0.41740761, 0.50061331, 0.58381902,
       0.66702473, 0.75023043, 0.83343614])

In [31]:
UM2_2o_pol

,bms,mwt,fishing_intensity,policy
0,0.0,0.25,0.000000,2 obs. RL at m. wt. 0.25
1,0.0,0.33,0.000000,2 obs. RL at m. wt. 0.33
2,0.0,0.41,0.000000,2 obs. RL at m. wt. 0.41
3,0.0,0.49,0.000000,2 obs. RL at m. wt. 0.49
4,0.0,0.57,0.021153,2 obs. RL at m. wt. 0.57
...,...,...,...,...
3995,3.5,0.49,0.109570,2 obs. RL at m. wt. 0.49
3996,3.5,0.57,0.141861,2 obs. RL at m. wt. 0.57
3997,3.5,0.65,0.177124,2 obs. RL at m. wt. 0.65
3998,3.5,0.73,0.189269,2 obs. RL at m. wt. 0.73
